# 🧪 Experimentation Engine: A/B Test Analysis Walkthrough

This notebook demonstrates a rigorous growth engineering workflow for analyzing A/B tests. We will walk through hypothesis definition, experimental design (power analysis), data simulation, and statistical inference.

## 🎯 Business Context

Our product team wants to improve the mobile landing page sign-up rate. They have proposed a change to the primary call-to-action button color from **Blue** to **Green**.

### Hypothesis

**If** we change the 'Sign Up' button color from Blue to Green on the mobile landing page  
**Then** we will see a 15% increase in conversion rate  
**Because** Green is culturally associated with 'Go' and financial success in our target market.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add src to path
sys.path.append(os.path.abspath(os.path.join('../')))

from src.simulator import ABTestSimulator, ExperimentConfig
from src.stats_engine import ABTestAnalyzer
from src import utils

# Configure plots
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [10, 6]
print("✅ Setup Complete")

## 1. Experimental Design & Power Analysis

Before running the test, we must calculate the required sample size to avoid Type II errors (missing a real effect). We use the following parameters:

- **Baseline Conversion Rate**: 8%
- **Minimum Detectable Effect (MDE)**: 15% relative lift (Target: 9.2%)
- **Significance Level (Alpha)**: 0.05
- **Statistical Power (1-Beta)**: 0.80

In [ ]:
config = ExperimentConfig()
simulator = ABTestSimulator(config)

print(f"Experiment Configuration:")
print(f"  Baseline Conversion: {config.baseline_conversion:.1%}")
print(f"  Minimum Detectable Effect: {config.relative_lift:.1%}")
print(f"  Required Sample Size: {simulator.sample_size:,} per variant")
print(f"  Total Sample Size Needed: {simulator.sample_size * 2:,}")

## 2. Data Generation / Loading

We will generate synthetic data that simulates the real experimental conditions.

In [ ]:
df, metadata = simulator.generate_with_metadata()
print(f"Generated {len(df):,} total records")
df.head()

## 3. Exploratory Data Analysis

Let's look at the raw counts and conversion rates by group.

In [ ]:
summary = df.groupby('group')['converted'].agg(['count', 'sum', 'mean']).reset_index()
summary.columns = ['Group', 'Total Users', 'Conversions', 'Conversion Rate']
summary['Conversion Rate Str'] = summary['Conversion Rate'].apply(lambda x: utils.format_percent(x))
summary

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='Group', y='Conversion Rate', data=summary, palette='viridis')
plt.title('Conversion Rate by Variant')
plt.ylabel('Conversion Rate (%)')
plt.ylim(0, 0.12)

# Add labels on top of bars
for i, row in summary.iterrows():
    plt.text(i, row['Conversion Rate'] + 0.005, f"{row['Conversion Rate']:.2%}", 
             ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 4. Statistical Analysis

We will perform a Two-Proportion Z-Test to determine if the observed lift is statistically significant.

In [ ]:
analyzer = ABTestAnalyzer()
result = analyzer.analyze(df)

print("=" * 40)
print("STATISTICAL ANALYSIS RESULTS")
print("=" * 40)
print(result)

### Visualizing Confidence Intervals

The 95% Confidence Interval for the lift shows the range where the true effect size likely falls.

In [ ]:
ci_low, ci_high = result.confidence_interval
lift = result.absolute_lift

plt.figure(figsize=(10, 4))
plt.errorbar(lift, 0, xerr=[[lift - ci_low], [ci_high - lift]], 
             fmt='o', color='green', markersize=10, capsize=10, elinewidth=3)
plt.axvline(0, color='red', linestyle='--', alpha=0.6)
plt.title('95% Confidence Interval for Absolute Lift')
plt.xlabel('Absolute Difference in Conversion Rate')
plt.yticks([])
plt.grid(axis='x', alpha=0.3)
plt.annotate(f"Lift: {lift:+.2%}", (lift, 0.05), ha='center')
plt.tight_layout()
plt.show()

## 5. Decision & Recommendation

Based on the statistical significance (p < 0.05) and the magnitude of the lift, we make a professional recommendation.

In [ ]:
recommendation = analyzer.generate_recommendation(result)
print(f"FINAL RECOMMENDATION: {recommendation}")

if recommendation == 'SHIP':
    print("✅ STATUS: High Confidence to Ship.")
    print("RATIONALE: The variant showed a statistically significant improvement ")
    print("that exceeds our Minimum Detectable Effect.")
else:
    print("⚠️ STATUS: Do Not Ship.")
    print("RATIONALE: The results were either non-significant or did not ")
    print("meet the required business impact.")